# 0 - Imports

In [ ]:
# !pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

In [ ]:
# os: Proporciona funciones para interactuar con el sistema operativo, como manejar rutas de archivos y directorios.
import os

# time: Permite acceder a funciones que manejan el tiempo, como pausas (sleep) o tomar el tiempo actual.
import time

# uuid: Usado para generar identificadores únicos universales (UUIDs), que son útiles para nombres de archivos únicos, por ejemplo.
import uuid

# cv2: OpenCV para operaciones relacionadas con visión por computadora, como procesamiento de imágenes y video.
import cv2
